In [1]:
# ! pip install sentence-splitter

In [2]:
# ! pip install transformers

In [3]:
# ! pip install SentencePiece

In [4]:
# !pip install torch

In [26]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'shah_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained('tuner007/pegasus_paraphrase')
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

---

## **Processing a paragraph of text**

In [27]:
# Paragraph of text
context = "To be clear: I’ve glamped before, and I was an asshole about it. We were invited to an AutoCamp property outside Yosemite National Park a couple years ago, but rather than focus on the sparkling white bathrooms, where I could comfortably bathe my tiny child, or the crisp bed, on which we read her stories about the granite monoliths we’d seen on the Valley floor, I lurked around the property and looked for opportunities to feel superior to my fellow glampers. One particularly shitty text to an old friend read: “I see a John Muir quote in yellow cursive neon lights. Peak irony.” My daughter, however, loved just about every minute of the trip and the comfort the amenities allowed my wife and I to relax into our first Yosemite trip as parents."

In [28]:
print(context)

To be clear: I’ve glamped before, and I was an asshole about it. We were invited to an AutoCamp property outside Yosemite National Park a couple years ago, but rather than focus on the sparkling white bathrooms, where I could comfortably bathe my tiny child, or the crisp bed, on which we read her stories about the granite monoliths we’d seen on the Valley floor, I lurked around the property and looked for opportunities to feel superior to my fellow glampers. One particularly shitty text to an old friend read: “I see a John Muir quote in yellow cursive neon lights. Peak irony.” My daughter, however, loved just about every minute of the trip and the comfort the amenities allowed my wife and I to relax into our first Yosemite trip as parents.


In [29]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['To be clear: I’ve glamped before, and I was an asshole about it.',
 'We were invited to an AutoCamp property outside Yosemite National Park a couple years ago, but rather than focus on the sparkling white bathrooms, where I could comfortably bathe my tiny child, or the crisp bed, on which we read her stories about the granite monoliths we’d seen on the Valley floor, I lurked around the property and looked for opportunities to feel superior to my fellow glampers.',
 'One particularly shitty text to an old friend read: “I see a John Muir quote in yellow cursive neon lights.',
 'Peak irony.”',
 'My daughter, however, loved just about every minute of the trip and the comfort the amenities allowed my wife and I to relax into our first Yosemite trip as parents.']

In [30]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

In [31]:
# This is the paraphrased text
paraphrase

[["I've done it before, and I was aasshole about it."],
 ['We were invited to a property that can serve several purposes, but rather than focus on the sparkling white bathroom, or the crisp bed, which we can comfortably bathe my child,'],
 ['In a text to an old friend, I see a John Muir quote in yellow cursive neon lights.'],
 ["It's peak irony."],
 ['My daughter loved every minute of the trip, and the comfort the amenities gave us allowed us to relax into our first Yosemite trip as parents, but']]

In [32]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

["I've done it before, and I was aasshole about it.",
 'We were invited to a property that can serve several purposes, but rather than focus on the sparkling white bathroom, or the crisp bed, which we can comfortably bathe my child,',
 'In a text to an old friend, I see a John Muir quote in yellow cursive neon lights.',
 "It's peak irony.",
 'My daughter loved every minute of the trip, and the comfort the amenities gave us allowed us to relax into our first Yosemite trip as parents, but']

In [33]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'"I\'ve done it before, and I was aasshole about it. We were invited to a property that can serve several purposes, but rather than focus on the sparkling white bathroom, or the crisp bed, which we can comfortably bathe my child, In a text to an old friend, I see a John Muir quote in yellow cursive neon lights. It\'s peak irony. My daughter loved every minute of the trip, and the comfort the amenities gave us allowed us to relax into our first Yosemite trip as parents, but"'

In [34]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(context)
print(paraphrased_text)


To be clear: I’ve glamped before, and I was an asshole about it. We were invited to an AutoCamp property outside Yosemite National Park a couple years ago, but rather than focus on the sparkling white bathrooms, where I could comfortably bathe my tiny child, or the crisp bed, on which we read her stories about the granite monoliths we’d seen on the Valley floor, I lurked around the property and looked for opportunities to feel superior to my fellow glampers. One particularly shitty text to an old friend read: “I see a John Muir quote in yellow cursive neon lights. Peak irony.” My daughter, however, loved just about every minute of the trip and the comfort the amenities allowed my wife and I to relax into our first Yosemite trip as parents.
"I've done it before, and I was aasshole about it. We were invited to a property that can serve several purposes, but rather than focus on the sparkling white bathroom, or the crisp bed, which we can comfortably bathe my child, In a text to an old fr

In [35]:
# paraphrased_text = paraphrased_text.replace("'", "")
paraphrased_text= paraphrased_text.strip('"')
print(paraphrased_text)

I've done it before, and I was aasshole about it. We were invited to a property that can serve several purposes, but rather than focus on the sparkling white bathroom, or the crisp bed, which we can comfortably bathe my child, In a text to an old friend, I see a John Muir quote in yellow cursive neon lights. It's peak irony. My daughter loved every minute of the trip, and the comfort the amenities gave us allowed us to relax into our first Yosemite trip as parents, but


In [71]:
#This is new addition
context = paraphrased_text

In [96]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer1 = AutoTokenizer.from_pretrained("prithivida/parrot_paraphraser_on_T5")

model1 = AutoModelForSeq2SeqLM.from_pretrained("prithivida/parrot_paraphraser_on_T5")

In [97]:
def get_responses(input_text,num_return_sequences):
  batch = tokenizer1.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model1.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer1.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [99]:
context = "Gardeners often find it hard to talk about their gardens. I'm a bit like that, though give me a cat audience and I'll expound eloquently on any gardening topic. A bit shy, and too much to say - I guess those are the reasons why I love writing about the garden so much. My gardening articles contain a mixture of garden realities and spiritual philosophies, personal memories, hopes for the future, showing my many gardening moods and my gardening ego (fragile at times). I've been working in my garden for eighteen years now. I'm always thinking about the garden, working or relaxing in it, or staring at the trees or borders out the house windows. I really am so very lucky, and I hope you enjoy reading the gardening articles I've written."

In [100]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['Gardeners often find it hard to talk about their gardens.',
 "I'm a bit like that, though give me a cat audience and I'll expound eloquently on any gardening topic.",
 'A bit shy, and too much to say - I guess those are the reasons why I love writing about the garden so much.',
 'My gardening articles contain a mixture of garden realities and spiritual philosophies, personal memories, hopes for the future, showing my many gardening moods and my gardening ego (fragile at times).',
 "I've been working in my garden for eighteen years now.",
 "I'm always thinking about the garden, working or relaxing in it, or staring at the trees or borders out the house windows.",
 "I really am so very lucky, and I hope you enjoy reading the gardening articles I've written."]

In [101]:
paraphrase_p = []

for i in sentence_list:
  a = get_responses(i,1)
  paraphrase_p.append(a)

In [102]:
paraphrase_p

[['Gardeners often find it hard to talk about their gardens.'],
 ["I'm a little like that, though give me a cat audience and I'll expound eloquently on any gardening topic."],
 ["A little shy and too much to say - I guess that's the reason why I love writing about the garden so much."],
 ['My gardening articles contain a mixture of garden realities and spiritual philosophies, personal memories, hopes for the future showing my many gardening moods and my gardening ego (fragile at times).'],
 ["I've been working in my garden for 18 years now."],
 ["I'm always thinking about the garden, working or relaxing in it or staring at the trees or the borders out the house windows."],
 ["I really am so lucky and I hope you enjoy reading the gardening articles I've written."]]

In [105]:
paraphrase2 = [' '.join(x) for x in paraphrase_p]
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
print(paraphrased_text)

"Gardeners often find it hard to talk about their gardens. I'm a little like that, though give me a cat audience and I'll expound eloquently on any gardening topic. A little shy and too much to say - I guess that's the reason why I love writing about the garden so much. My gardening articles contain a mixture of garden realities and spiritual philosophies, personal memories, hopes for the future showing my many gardening moods and my gardening ego (fragile at times). I've been working in my garden for 18 years now. I'm always thinking about the garden, working or relaxing in it or staring at the trees or the borders out the house windows. I really am so lucky and I hope you enjoy reading the gardening articles I've written."
